In [1]:
!pip install --upgrade spacy
!python -m spacy download it_core_news_sm
!pip install --upgrade nltk

     |████████████████████████████████| 12.8MB 263kB/s 
     |████████████████████████████████| 9.1MB 22.9MB/s 
     |████████████████████████████████| 460kB 39.6MB/s 
     |████████████████████████████████| 1.1MB 34.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 122kB 50.9MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=7d5145af98af23fce164fbcd4edb5b3c8f4ef4a23e3859ce574eb75d810e57a7
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0

In [2]:
import tensorflow as tf
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# path = tf.keras.utils.get_file(
# 'nietzsche.txt',
# origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Ql23NJjVNA0V31EvoiaCYdrBHTnGqRgX' -O complete_dataset.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=16M7ZFtSByUQ1OWczjIajCAMua89mXgfq' -O testing.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YKoi3m1W7b-_w8LS3bQF3Ps3EK9ha7QR' -O testing_min_tokens_8.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qmv3PGq-R1Ao-DrjT2oFozSRrrVfBmGT' -O training.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XWUfOmzB1CmjxEF03sg8DoF3u8BoArni' -O validation.txt

--2021-05-24 15:02:52--  https://docs.google.com/uc?export=download&id=1Ql23NJjVNA0V31EvoiaCYdrBHTnGqRgX
Resolving docs.google.com (docs.google.com)... 173.194.215.139, 173.194.215.102, 173.194.215.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pfhhflvl45dokl9jm6bc8r6d9sdqdaop/1621868550000/03570075665403880665/*/1Ql23NJjVNA0V31EvoiaCYdrBHTnGqRgX?e=download [following]
--2021-05-24 15:02:53--  https://doc-0s-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pfhhflvl45dokl9jm6bc8r6d9sdqdaop/1621868550000/03570075665403880665/*/1Ql23NJjVNA0V31EvoiaCYdrBHTnGqRgX?e=download
Resolving doc-0s-3o-docs.googleusercontent.com (doc-0s-3o-docs.googleusercontent.com)... 173.194.218.132, 2607:f8b0:400c:c14::84
Connecting to doc-0s-3o-docs.googleusercontent.com (doc-0s

# TRAINING E VALIDATION

In [4]:
complete_dataset_path = 'complete_dataset.txt'
train_path = 'training.txt'
validation_path = 'validation.txt'

In [5]:
text = open(train_path).read()
print(len(text))
print(text[:500])

4402103
"Quanto ho da dirgli è destinato esclusivamente alle sue orecchie."
"Me l'ha detto Ibun, l'altro mio figlio".
Allocco si avvicinò una sedia e vi si sedette a cavalcioni, appoggiando le braccia incrociate e il mento sulla spalliera.
"Brick ha ragione, vero?"
Cuoci le patate, tagliale a dadi.
Presto Bilbo poté dar loro qualche rapida occhiata mentre il buio si infittiva.
Passò fra i tavoli e vide i tre Malfoy, stretti come se non sapessero se star lì o no, ma nessuno vi faceva caso.
Attento a vede


In [6]:
text_val = open(validation_path).read()
print(len(text_val))
print(text_val[:500])

552034
Harry aveva lo stomaco chiuso.
Ma non credere che l'abbiano fatto per buon cuore.
Proprio come aveva predetto Gandalf, vedete.
"Ma certo, mio Signore" disse Lucius Malfoy.
i loro corpi venivano calpestati dalla folla in ritirata.
In quel preciso istante entrò Madama Chips e li mise alla porta, dicendo che Ron aveva bisogno di dormire.
Allora si videro le molte barche punteggiare scure la superficie del lago, e il vento portò le voci del popolo di Esgaroth che piangeva la perdita della città, dei


In [ ]:
"""


"Quanto ho da dirgli è destinato esclusivamente alle sue orecchie."
"Me l'ha detto Ibun, l'altro mio figlio".
"Brick ha ragione, vero?"
Cuoci le patate, tagliale a dadi.
Presto Bilbo poté dar loro qualche rapida occhiata mentre il buio si infittiva.
Attento a vedere, ma senza esporsi.
"Credo proprio che dovremmo".
"Che c'è?"


"""

In [6]:
maxlen = 64
step = 32
sentences = []
next_chars = []

for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])

In [8]:
print(sentences[0:2])

['"Quanto ho da dirgli è destinato esclusivamente alle sue orecchi', ' esclusivamente alle sue orecchie."\n"Me l\'ha detto Ibun, l\'altro']


In [9]:
print(len(sentences))
for i in range(3):
  print(sentences[i])
  print()

137564
"Quanto ho da dirgli è destinato esclusivamente alle sue orecchi

 esclusivamente alle sue orecchie."
"Me l'ha detto Ibun, l'altro

e."
"Me l'ha detto Ibun, l'altro mio figlio".
Allocco si avvicin



In [10]:
sentences_val = []
next_chars_val = []

for i in range(0, len(text_val)-maxlen, step):
    sentences_val.append(text_val[i:i+maxlen])
    next_chars_val.append(text_val[i+maxlen])

In [11]:
sentences[0]

'"Quanto ho da dirgli è destinato esclusivamente alle sue orecchi'

In [12]:
next_chars[0]

'e'

In [7]:
chars = sorted(list(set(text)))
vocab_len = len(chars)

In [14]:
print(len(chars))
print(chars)

111
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', 'º', 'À', 'Á', 'È', 'É', 'Ì', 'Ò', 'Ù', 'ß', 'à', 'á', 'â', 'ä', 'è', 'é', 'ê', 'ë', 'ì', 'î', 'ï', 'ò', 'ó', 'ô', 'ö', 'ù', 'û', 'ü', '‘', '…']


In [8]:
char_indices = dict((char, chars.index(char)) for char in chars)
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 '*': 8,
 '+': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 ';': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'X': 50,
 'Y': 51,
 'Z': 52,
 ']': 53,
 '_': 54,
 'a': 55,
 'b': 56,
 'c': 57,
 'd': 58,
 'e': 59,
 'f': 60,
 'g': 61,
 'h': 62,
 'i': 63,
 'j': 64,
 'k': 65,
 'l': 66,
 'm': 67,
 'n': 68,
 'o': 69,
 'p': 70,
 'q': 71,
 'r': 72,
 's': 73,
 't': 74,
 'u': 75,
 'v': 76,
 'w': 77,
 'x': 78,
 'y': 79,
 'z': 80,
 '{': 81,
 'º': 82,
 'À': 83,
 'Á': 84,
 'È': 85,
 'É': 86,
 'Ì': 87,
 'Ò': 88,
 'Ù': 89,
 'ß': 90,
 'à': 91,
 'á': 92,
 'â': 93,
 'ä': 94,
 'è': 95,
 'é': 96,
 'ê': 97,
 'ë': 98,
 'ì': 99,
 'î': 100

In [16]:
chars_val = sorted(list(set(text_val)))
vocab_len_val = len(chars_val)

In [17]:
print(len(chars))
print(len(chars_val))

111
100


In [18]:
char_indices_val = dict((char, chars_val.index(char)) for char in chars_val)
char_indices_val

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 '+': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '1': 12,
 '2': 13,
 '4': 14,
 '5': 15,
 '7': 16,
 '9': 17,
 ':': 18,
 ';': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'X': 44,
 'Y': 45,
 'Z': 46,
 '[': 47,
 ']': 48,
 '_': 49,
 '`': 50,
 'a': 51,
 'b': 52,
 'c': 53,
 'd': 54,
 'e': 55,
 'f': 56,
 'g': 57,
 'h': 58,
 'i': 59,
 'j': 60,
 'k': 61,
 'l': 62,
 'm': 63,
 'n': 64,
 'o': 65,
 'p': 66,
 'q': 67,
 'r': 68,
 's': 69,
 't': 70,
 'u': 71,
 'v': 72,
 'w': 73,
 'x': 74,
 'y': 75,
 'z': 76,
 '°': 77,
 'À': 78,
 'È': 79,
 'Ù': 80,
 'à': 81,
 'á': 82,
 'â': 83,
 'ä': 84,
 'ç': 85,
 'è': 86,
 'é': 87,
 'ê': 88,
 'ë': 89,
 'ì': 90,
 'î': 91,
 'ï': 92,
 'ò': 93,
 'ó': 94,
 'ö': 95,
 'ù': 96,
 'û': 97,
 '‘': 98,
 '…': 99}

In [19]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)               # Matrice tridimensionale di dimensione len(sentences) x maxlen x len(chars), in pratica per ogni carattere di ogni frase, questa matrice ci dice quale viene utilizzato
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)                       # Matrice bidimensionale di dimensione len(sentences) len(chars)
# x_test = np.zeros((4, 3, 2), dtype=np.bool)                                   # Facciamo finta di avere in totale 4 frasi, ognuna di lunghezza 3, e per cui ci sia un vocabolario di 2 caratteri
# y_test = np.zeros((4, 2), dtype=np.bool)

In [20]:
print(len(sentences), maxlen, len(chars))
# print('x\n',x_test)
# print('y\n',y_test)

137564 64 111


In [21]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [22]:
x_val= np.zeros((len(sentences_val), maxlen, len(chars)), dtype=np.bool)
y_val = np.zeros((len(sentences_val), len(chars)), dtype=np.bool)


for i, sentence in enumerate(sentences_val):
    for t, char in enumerate(sentence):
        x_val[i, t, char_indices_val[char]] = 1
    y_val[i, char_indices_val[next_chars_val[i]]] = 1


# train_sentences = []
# val_sentences = []


# inputs = train_sentences
# outputs = val_sentences

# model = tf.keras.Model(inputs=inputs, outputs=outputs)

# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# # Preprocess the data (these are NumPy arrays)
# x_train = x_train.reshape(60000, 784).astype("float32") / 255
# x_val = x_test.reshape(10000, 784).astype("float32") / 255

# y_train = y_train.astype("float32")
# y_val = y_test.astype("float32")

In [23]:
y[100]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [24]:
y_val[100]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [25]:
x[100].shape

(64, 111)

In [26]:
x_val[100].shape

(64, 111)

In [27]:
print(x.shape)
print(y.shape)

(137564, 64, 111)
(137564, 111)


In [28]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential()
model.add(layers.LSTM(vocab_len, input_shape=(maxlen, vocab_len)))

In [29]:
model.add(layers.Dense(vocab_len, activation="softmax"))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 111)               99012     
_________________________________________________________________
dense (Dense)                (None, 111)               12432     
Total params: 111,444
Trainable params: 111,444
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [21]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
timestamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
print(timestamp)

2021_05_24-14_56_29


In [ ]:
!mkdir "/content/drive/My Drive/OdyssAI/Models_RNN/$timestamp"

In [ ]:
import random
import sys

CHECKPOINT = 25


for epoch in range(1, 201):
    print("Epoch ", epoch)
    model.fit(x, y, batch_size=128, validation_data=(x_val, y_val))
    
    if (epoch) % CHECKPOINT == 0:
      start_index = random.randint(0, len(text) - maxlen - 1)
      generated_text = text[start_index:start_index + maxlen]
      f = open("epoch_{}.txt".format(epoch), "w")
      f.write("Epoch {}\n".format(epoch))
      print('--- Generating with seed: "' + generated_text + '"')
      f.write("\nSeed\n{}\n".format(generated_text))
      for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        f.write("\nTemperature {}\n".format(temperature))
        # print(generated_text)
        for i in range(300):
          sampled = np.zeros((1, maxlen, len(chars)))
          for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.
          preds = model.predict(sampled, verbose=0)[0]
          next_index = sample(preds, temperature)
          next_char = chars[next_index]
          generated_text += next_char
          generated_text = generated_text[1:]
          # sys.stdout.write(next_char)
          f.write(next_char)
      f.close()
      filename = "epoch_{}.txt".format(epoch)
      
      !cp $filename "/content/drive/My Drive/OdyssAI/Models_RNN/$timestamp"

In [ ]:
model.save('odyssai_rnn')

In [ ]:
!zip -r OdyssAI_RNN.zip "odyssai_rnn"

In [ ]:
!cp OdyssAI_RNN.zip "/content/drive/My Drive/OdyssAI/Models_RNN/$timestamp"

# Testing

In [10]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-JhQoWqVvHZxAxI6NDnadoNmFyRO1lAQ' -O OdyssAI_RNN.zip

--2021-05-24 15:02:59--  https://docs.google.com/uc?export=download&id=1-JhQoWqVvHZxAxI6NDnadoNmFyRO1lAQ
Resolving docs.google.com (docs.google.com)... 173.194.213.102, 173.194.213.138, 173.194.213.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.213.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/31ft8mi6pdis6ipsh0impnsqgmmj2jlv/1621868550000/03570075665403880665/*/1-JhQoWqVvHZxAxI6NDnadoNmFyRO1lAQ?e=download [following]
--2021-05-24 15:03:01--  https://doc-0g-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/31ft8mi6pdis6ipsh0impnsqgmmj2jlv/1621868550000/03570075665403880665/*/1-JhQoWqVvHZxAxI6NDnadoNmFyRO1lAQ?e=download
Resolving doc-0g-3o-docs.googleusercontent.com (doc-0g-3o-docs.googleusercontent.com)... 173.194.218.132, 2607:f8b0:400c:c14::84
Connecting to doc-0g-3o-docs.googleusercontent.com (doc-0g

In [11]:
!unzip 'OdyssAI_RNN.zip'

Archive:  OdyssAI_RNN.zip
   creating: odyssai_rnn/
   creating: odyssai_rnn/variables/
  inflating: odyssai_rnn/variables/variables.index  
  inflating: odyssai_rnn/variables/variables.data-00000-of-00001  
  inflating: odyssai_rnn/saved_model.pb  
   creating: odyssai_rnn/assets/


In [12]:
model = tf.keras.models.load_model('odyssai_rnn/')

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               122880    
_________________________________________________________________
dense (Dense)                (None, 111)               14319     
Total params: 137,199
Trainable params: 137,199
Non-trainable params: 0
_________________________________________________________________


In [14]:
from scipy import stats

def fileStringToList(filename):
  f = open(filename, 'r')
  lines = f.readlines()
  f.close()
  line = lines[1].replace('[', '').replace(']\n', '')
  score_list = line.split(', ')

  return score_list
  
def getScoresChkpnt(bleu_file, len_diff_file):
  bleu_list = fileStringToList(bleu_file)
  len_diff_list = fileStringToList(len_diff_file)
  bleu_list = list(np.float_(bleu_list))
  len_diff_list = list(np.int_(len_diff_list))

  return bleu_list, len_diff_list

def writeOnFile (filename, input_list):
  input_file = open(filename, 'w+')
  input_file.write('Score:\n' + str(input_list) + '\n\n' +
              'Mean: ' + str(np.mean(input_list)) + '\n' +
              'Median: ' + str(np.median(input_list)) + '\n' +
              'Mode: ' + str(stats.mode(input_list)[0][0]) + '\t\t\tCount: ' + str(stats.mode(input_list)[1][0]) + '\n' +
              'Max: ' + str(np.max(input_list)) + '\t\t\tMin: ' + str(np.min(input_list)))
  input_file.close()
  
"""Adatta la grandezza dell'input alle dimensioni della finestra e genera testo tramite il modello"""
def rnnGenerateText (seed, generate_len):                                   
  temperature = 0.5
  generated_text = seed
  # generated_text = '"A momenti si stanca di andarle dietro, agitando le mani e strillando... scusa, Giovanna".\nGeralt lanciò pietre come un mostro.\n'
  if len(generated_text) >= maxlen:
    generated_text = generated_text[:maxlen]
    output = generated_text
  else:
    output = generated_text + " "
    generated_text = generated_text + (" " * (maxlen - len(generated_text)))
  # print(f"\nSeed: {output}\n")
  # output = generated_text
  for i in range(generate_len):
    # sampled = np.zeros((1, len(generated_text), len(chars)))
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    output += next_char
    generated_text = generated_text[1:]
  # print(type(output), output)
  return output

In [40]:
!mkdir -p "/content/drive/My Drive/OdyssAI/Models_RNN/BLEU"
!mkdir -p "/content/drive/My Drive/OdyssAI/Models_RNN"

In [17]:
# f = open('bleu_scores_prompt4.txt', 'r')
"""Oggetto usato per calcolare il BLEU score"""
class ComputeBleu:
  def __init__(self, testfile, min_prompt, max_prompt, checkpoint, model = None):
    self.tf = open(testfile, 'r')
    self.min_prompt = min_prompt
    self.max_prompt = max_prompt
    self.checkpoint = checkpoint
    self.model = model

  """Metodo che calcola il BLEU, fa parte della classe ComputeBleu"""
  def computeBleu(self, scores = [], len_diffs = []):
    lines = self.tf.readlines()
    self.tf.close()
    nlp = spacy.load('it_core_news_sm')
    cc = SmoothingFunction()
    for prompt_size in range(self.min_prompt, self.max_prompt):
      for line_counter in range(len(scores), len(lines)):
        reference = [[]]
        hypotesis = []
        print(f'Line {line_counter+1}')
        ref_doc = nlp(lines[line_counter].replace('\n', ''))
        for ref_doc_counter, token in enumerate(ref_doc):
          reference[0].append(str(token))
        print(f'Reference: {ref_doc_counter + 1}\t{reference}')
        prompt = str(ref_doc[0:prompt_size])
        hyp_doc = nlp(rnnGenerateText(prompt, len(lines[line_counter])))
        for hyp_doc_counter, token in enumerate(hyp_doc):
          hypotesis.append(str(token))
        print(f'Hypotesis: {hyp_doc_counter + 1}\t{hypotesis}')
        len_diff = ref_doc_counter - hyp_doc_counter
        print(f'Length difference: {len_diff}')
        score = sentence_bleu(reference, hypotesis, smoothing_function=cc.method4)
        print(f'BLEU SCORE: {score}')
        print(100 * '-')

        scores.append(score)
        len_diffs.append(ref_doc_counter - hyp_doc_counter)
        if (line_counter + 1) % self.checkpoint == 0:
          writeOnFile(f'bleu_scores_prompt{prompt_size}_{line_counter+1}.txt', scores)
          writeOnFile(f'length_difference_prompt{prompt_size}_{line_counter+1}.txt', len_diffs)
          !cp bleu_scores_prompt{prompt_size}_{line_counter+1}.txt "/content/drive/My Drive/OdyssAI/Models_RNN"
          !cp length_difference_prompt{prompt_size}_{line_counter+1}.txt "/content/drive/My Drive/OdyssAI/Models_RNN"
          
      writeOnFile(f'bleu_scores_prompt{prompt_size}.txt', scores)
      writeOnFile(f'length_difference_prompt{prompt_size}.txt', len_diffs)
      !cp bleu_scores_prompt{prompt_size}.txt "/content/drive/My Drive/OdyssAI/Models_RNN/BLEU"
      !cp length_difference_prompt{prompt_size}.txt "/content/drive/My Drive/OdyssAI/Models_RNN/BLEU"

      scores = []
      len_diffs = []

In [19]:
# GET CHECKPOINTS
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-fbvR_dKA_HiwaaS9yNPqVJzwhDPEPrj' -O bleu_scores_prompt3_4800.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-l75GMEF8paP3BuCS2XEhV5UA12xa0wN' -O length_difference_prompt3_4800.txt

--2021-05-24 15:03:54--  https://docs.google.com/uc?export=download&id=1-fbvR_dKA_HiwaaS9yNPqVJzwhDPEPrj
Resolving docs.google.com (docs.google.com)... 108.177.11.138, 108.177.11.101, 108.177.11.139, ...
Connecting to docs.google.com (docs.google.com)|108.177.11.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qlnae4in66l96ri9qteem2ul4q9nd7un/1621868625000/03570075665403880665/*/1-fbvR_dKA_HiwaaS9yNPqVJzwhDPEPrj?e=download [following]
--2021-05-24 15:03:54--  https://doc-0s-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qlnae4in66l96ri9qteem2ul4q9nd7un/1621868625000/03570075665403880665/*/1-fbvR_dKA_HiwaaS9yNPqVJzwhDPEPrj?e=download
Resolving doc-0s-3o-docs.googleusercontent.com (doc-0s-3o-docs.googleusercontent.com)... 173.194.218.132, 2607:f8b0:400c:c14::84
Connecting to doc-0s-3o-docs.googleusercontent.com (doc-0s-3o-

In [20]:
# COMPUTE BLEU
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from os import path
import spacy

bleu = ComputeBleu('testing_min_tokens_8.txt', 6, 7, 800)
score_files = getScoresChkpnt('bleu_scores_prompt3_4800.txt', 'length_difference_prompt3_4800.txt')

bleu.computeBleu(score_files[0], score_files[1])

Line 4801
Reference: 10	[['Cahir', 'era', 'stato', 'inviato', 'a', 'Thanedd', 'in', 'missione', 'speciale', ':']]


TypeError: ignored

In [ ]:
# COMPUTE BLEU
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from os import path
import spacy

bleu = ComputeBleu('testing_min_tokens_8.txt', 1, 2, 800)
score_files = getScoresChkpnt('bleu_scores_prompt1_4000.txt', 'length_difference_prompt1_4000.txt')

bleu.computeThisShit(score_files[0], score_files[1])

In [ ]:
# RESTART BLEU FROM CHECKPOINT
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from os import path
import spacy

checkpoint = 800                                                                # After how many lines a checkpoint is saved
min_prompt = 4                                                                  # Minimum prompt size (in tokens) min_prompt = n (n INCLUDED)
max_prompt = 5                                                                  # Maximum prompt size (in tokens) max_prompt = m (m EXCLUDED)

nlp = spacy.load('it_core_news_sm')
f = open('testing_min_tokens_8.txt', 'r')
lines = f.readlines()
f.close()

cc = SmoothingFunction()

In [ ]:
# BLEU SCORE

CHECKPOINT = 800                                                                # After how many lines a checkpoint is saved
min_prompt = 1                                                                  # Minimum prompt size (in tokens) min_prompt = n (n INCLUDED)
max_prompt = 2                                                                  # Maximum prompt size (in tokens) max_prompt = m (m EXCLUDED)
nlp = spacy.load('it_core_news_sm')

f = open('testing_min_tokens_8.txt', 'r')
lines = f.readlines()
f.close()

cc = SmoothingFunction()

for prompt_size in range(min_prompt, max_prompt):

  scores = []
  len_diffs = []

  for line_counter, line in enumerate(lines):
    reference = [[]]
    hypotesis = []
    print(f'Line {line_counter+1}')
    ref_doc = nlp(line.replace('\n', ''))

    for ref_doc_counter, token in enumerate(ref_doc):
      reference[0].append(str(token))
    
    print(f'Reference: {ref_doc_counter + 1}\t{reference}')
    prompt = str(ref_doc[0:prompt_size])
    

    hyp_doc = nlp(rnnGenerateText(prompt, len(line)))

    # hyp_doc = textManipulation(output)

    for hyp_doc_counter, token in enumerate(hyp_doc):
      hypotesis.append(str(token))

    print(f'Hypotesis: {hyp_doc_counter + 1}\t{hypotesis}')
    len_diff = ref_doc_counter - hyp_doc_counter
    print(f'Length difference: {len_diff}')
    score = sentence_bleu(reference, hypotesis, smoothing_function=cc.method4)
    print(f'BLEU SCORE: {score}')
    print(100 * '-')
    scores.append(score)
    len_diffs.append(ref_doc_counter - hyp_doc_counter)

    if (line_counter + 1) % CHECKPOINT == 0:
      f = open(f'bleu_scores_prompt{prompt_size}_{line_counter+1}.txt', 'w+')
      f1 = open(f'length_difference_prompt{prompt_size}_{line_counter+1}.txt', 'w+')
      writeOnFile(f, scores)
      writeOnFile(f1, len_diffs)
      f.close()
      f1.close()
      !cp bleu_scores_prompt{prompt_size}_{line_counter+1}.txt "/content/drive/My Drive/OdyssAI/Models_RNN"
      !cp length_difference_prompt{prompt_size}_{line_counter+1}.txt "/content/drive/My Drive/OdyssAI/Models_RNN"

  f = open(f'bleu_scores_prompt{prompt_size}.txt', 'w+')
  f1 = open(f'length_difference_prompt{prompt_size}.txt', 'w+')
  writeOnFile(f, scores)
  writeOnFile(f1, len_diffs)
  f.close()
  f1.close()
  # while not path.exists(f"/content/drive/My Drive/OdyssAI/Models/BLEU/bleu_scores_prompt{prompt_size}.txt"):
  !cp bleu_scores_prompt{prompt_size}.txt "/content/drive/My Drive/OdyssAI/Models_RNN/BLEU"
  # while not path.exists(f"/content/drive/My Drive/OdyssAI/Models/BLEU/length_difference_prompt{prompt_size}.txt"):
  !cp length_difference_prompt{prompt_size}.txt "/content/drive/My Drive/OdyssAI/Models_RNN/BLEU"

In [26]:
lines = ["Le viscere gli si strinsero come un pugno.",
         "E lì essa rimase, che Eol se la prese in moglie, e lungo tempo passò prima che alcuno della sua stirpe ne avesse nuovamente notizia.",
         "Io non so cosa veramente sia svenire;",
         "Se non ti do qualche consiglio non te la cavi, questo è quanto io posso dire.",
         "Mi ha sempre incuriosito, strigo, perché la gente abbia tanta paura di voi."]

for line in lines:
  print("Prompt:", line[:maxlen])
  sampleText = rnnGenerateText(line, 50)
  print("\nSample:", sampleText)
  print("\n")

Prompt: Le viscere gli si strinsero come un pugno.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until



Sample: Le viscere gli si strinsero come un pugno. di spiegrette.
"Imentra ribastare i suoi di Riduli


Prompt: E lì essa rimase, che Eol se la prese in moglie, e lungo tempo p

Sample: E lì essa rimase, che Eol se la prese in moglie, e lungo tempo perguate della vendia sulla giolia di Riva.
"Era un


Prompt: Io non so cosa veramente sia svenire;

Sample: Io non so cosa veramente sia svenire; erate una striuna anica, chiesi nella lora era di 


Prompt: Se non ti do qualche consiglio non te la cavi, questo è quanto i

Sample: Se non ti do qualche consiglio non te la cavi, questo è quanto i capello so maghio Pottars, non so che cosa sì."
R


Prompt: Mi ha sempre incuriosito, strigo, perché la gente abbia tanta pa

Sample: Mi ha sempre incuriosito, strigo, perché la gente abbia tanta parte segrete di sconsere della persa a salsi troppo


